In [15]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
import torchvision
from torchvision import models as mods
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, FastICA
from sklearn.covariance import LedoitWolf, MinCovDet

from torchvision.transforms import transforms

from sklearn.metrics import roc_auc_score
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor
from sklearn.random_projection import SparseRandomProjection

import torch
import torch.nn as nn

from data.mvtec import *
inv_norm = MVTEC().inv_normalize

from hugeica import *
from models.backbones import *

np.random.seed(252525)
torch.manual_seed(252525)

MVTEC_PATH="/home/ios/data/mvtec"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
MVTEC.CLASSES

['bottle',
 'carpet',
 'leather',
 'pill',
 'tile',
 'wood',
 'cable',
 'grid',
 'toothbrush',
 'zipper',
 'capsule',
 'hazelnut',
 'metal_nut',
 'screw',
 'transistor']

In [16]:
bs = 50
p = 5 # fallback patch_size
s = 1 # stride

net = mods.efficientnet_b4(pretrained=True).features
net = net.to(device)
net.eval()

for clazz in [14]:
    
    X_, X_valid_, X_test_, X_labels_, T = dataloader(clazz, P=224, s=224, label_per_patch=False, load_size=256, crop_size=224, normalize=False,
                                                     MVTEC_PATH=MVTEC_PATH )
    
    X_test_images = X_test_

    X = concat_features(X_, net, layers=[5,6], fmap_pool=False)
    X_valid = concat_features(X_valid_, net, layers=[5,6], fmap_pool=False)
    X_test = concat_features(X_test_, net, layers=[5,6], fmap_pool=False)

    shapes = (X.shape, X_test.shape, X_valid.shape)
    c = X.shape[1]
    t = X.shape[2] # tile_size
    n = len(X_valid) + len(X_test)
    print(shapes)
    
    for i,j in enumerate(range(1, len(X), 5)):
    
        model = SFA(shape=(c, t, t), 
                        BSZ=(p, p), 
                        stride=s, 
                        n_components="q9999",
                        remove_components=0,
                        max_components=5000,
                        min_components=10,
                        mode="ta")

        model.fit(X[:j], 1, bs=5000, lr=1e-3, logging=-10)

        #auc_g, tg       = auc_global_mean_shift(model, X_valid, X_test)
        auc_l, tl       = auc_local_mean_shift(model, X, X_valid, X_test, size=2)
        #auc_c, tc, cset = auc_cluster_mean_shift(model, X, X_valid, X_test, True, return_coreset=True)
        #auc_cc, tcc     = auc_cluster_mean_shift(model, X, X_valid, X_test, False, k=len(cset))
        #auc_s, ts       = auc_sources_mean_shift(model, X, X_valid, X_test)
        #auc_cs, tcs     = auc_coreset(X, X_valid, X_test)

        #df = pd.DataFrame(np.asarray([clazz, auc_g, auc_l, auc_c, auc_cc, auc_s, n/tg, n/tl, n/tc, n/tcc, n/ts, p, t, model.model.n_components])[:, None].T, 
        #                  columns=["class", "AUC-g", "AUC-l", "AUC-c", "AUC-cc", "AUC-s", "fps-g", "fps-l", "fps-c", "fps-cc", "fps-s", "p_size", "t", "n_components"]) 

        map_i, map_o = local_mean_shift(model, X, X_valid, X_test, size=2)
        anomaly_map = make_anomaly_map(map_i, map_o)[1]
        anomaly_map = anomaly_map + X_test_images
        

        im_test  = torchvision.utils.make_grid(torch.from_numpy(anomaly_map), nrow=10, normalize=True,  value_range=(0,  1), rscale_each=False , padding=3 )

        n_rows_test = len(X_test_images)//10
        n_cols_train = 1 + len(X_)//n_rows_test
        if j < n_cols_train:
            trn = np.concatenate([X_[:j], np.zeros_like(X_[:1]).repeat(n_cols_train - j, axis=0)], axis=0)
        else:
            trn = X_[:j]
        im_train = torchvision.utils.make_grid(torch.from_numpy(trn), nrow=n_cols_train, normalize=True,  value_range=(0,  1), rscale_each=False , padding=3 )

        margin = 100
        im_train_ = np.zeros((3, im_test.shape[1], im_train.shape[2] + margin ))
        im_train_[:, :im_train.shape[1], margin:margin+im_train.shape[2]] = im_train

        f = plt.figure(figsize=(15,5)) #figure with correct aspect ratio
        ax = plt.axes((0,0,1,1)) #axes over whole figure
        ax.imshow(np.concatenate([im_test, im_train_], axis=2).transpose(1,2,0))
        x,y = im_test.shape[2] + im_train_.shape[2] - 550, im_train_.shape[1] - 100
        ax.text(x,y, f"AUC = {auc_l:.2f}", bbox=dict(fill="white", edgecolor='red', linewidth=2), size="medium")
        ax.axis('off')
        _ = f.savefig(f"im_{clazz}_{i:03d}.png",dpi=300)

KeyboardInterrupt: 

In [14]:
!ffmpeg -r 2 -i im_13_%3d.png -c:v libx264 -vf fps=1 -pix_fmt yuv420p out.mp4

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ios/anaconda3/envs/pytorch3 --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    

In [ ]:
bs = 50
p = 4 # fallback patch_size
s = 1 # stride

net = mods.efficientnet_b4(pretrained=True).features
net = net.to(device)
net.eval()

for clazz in [13]:
    
    X_, X_valid_, X_test_, X_labels_, T = dataloader(clazz, P=224, s=224, label_per_patch=False, load_size=256, crop_size=224, normalize=False)

    X = concat_features(X_, net, layers=[5,6], fmap_pool=False)
    X_valid = concat_features(X_valid_, net, layers=[5,6], fmap_pool=False)
    X_test = concat_features(X_test_, net, layers=[5,6], fmap_pool=False)

    shapes = (X.shape, X_test.shape, X_valid.shape)
    c = X.shape[1]
    t = X.shape[2] # tile_size
    n = len(X_valid) + len(X_test)
    print(shapes)
    
    model = SFA(shape=(c, t, t), 
                    BSZ=(p, p), 
                    stride=s, 
                    n_components="q9999",
                    remove_components=0,
                    max_components=5000,
                    min_components=100,
                    mode="ta")

    model.fit(X, 1, bs=5000, lr=1e-3, logging=-10)
    
    auc_g, tg       = auc_global_mean_shift(model, X_valid, X_test)
    auc_l, tl       = auc_local_mean_shift(model, X, X_valid, X_test, size=2)
    auc_c, tc, cset = auc_cluster_mean_shift(model, X, X_valid, X_test, True, return_coreset=True)
    auc_cc, tcc     = auc_cluster_mean_shift(model, X, X_valid, X_test, False, k=len(cset))
    auc_s, ts       = auc_sources_mean_shift(model, X, X_valid, X_test)
    #auc_cs, tcs     = auc_coreset(X, X_valid, X_test)

    df = pd.DataFrame(np.asarray([clazz, auc_g, auc_l, auc_c, auc_cc, auc_s, n/tg, n/tl, n/tc, n/tcc, n/ts, p, t, model.model.n_components])[:, None].T, 
                      columns=["class", "AUC-g", "AUC-l", "AUC-c", "AUC-cc", "AUC-s", "fps-g", "fps-l", "fps-c", "fps-cc", "fps-s", "p_size", "t", "n_components"]) 
    
    plt.rcParams["figure.figsize"] = (25, 25)
    map_i, map_o = local_mean_shift(model, X, X_valid, X_test, size=2)
    anomaly_map = make_anomaly_map(map_i, map_o)[1]
    anomaly_map = anomaly_map + X_test_images
    show( torchvision.utils.make_grid(torch.from_numpy(anomaly_map), nrow=10, normalize=True,  value_range=(0,  1), rscale_each=False , padding=3 ) )